In [ ]:
import pandas as pd
import numpy as np

loading the dataset

In [ ]:
orig_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/healthcare-dataset-stroke-data.csv")
orig_data = orig_data.drop("id", axis = 1)
orig_data = orig_data.dropna()
orig_data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [ ]:
orig_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4909 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4909 non-null   object 
 1   age                4909 non-null   float64
 2   hypertension       4909 non-null   int64  
 3   heart_disease      4909 non-null   int64  
 4   ever_married       4909 non-null   object 
 5   work_type          4909 non-null   object 
 6   Residence_type     4909 non-null   object 
 7   avg_glucose_level  4909 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     4909 non-null   object 
 10  stroke             4909 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 460.2+ KB


In [ ]:
print(len(orig_data.query("stroke == 1")))
print(len(orig_data.query("stroke == 0")))

209
4700


In [ ]:
#separating the dataset which has less number of representation in the dataset
data_stroke = orig_data.query("stroke == 1")
data_stroke.head()
# print(type(data_stroke))
print(data_stroke.shape)

(209, 11)


In [ ]:
!pip install sdv

In [ ]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()

**creating Metadata** </br>
Metadata provides context and describes the characteristics, attributes, and properties of the actual data, helping to understand and manage the data effectively

In [ ]:
metadata.detect_from_dataframe(data=data_stroke)

In [ ]:
python_dict = metadata.to_dict()

In [ ]:
python_dict

{'columns': {'gender': {'sdtype': 'categorical'},
  'age': {'sdtype': 'numerical'},
  'hypertension': {'sdtype': 'numerical'},
  'heart_disease': {'sdtype': 'numerical'},
  'ever_married': {'sdtype': 'categorical'},
  'work_type': {'sdtype': 'categorical'},
  'Residence_type': {'sdtype': 'categorical'},
  'avg_glucose_level': {'sdtype': 'numerical'},
  'bmi': {'sdtype': 'numerical'},
  'smoking_status': {'sdtype': 'categorical'},
  'stroke': {'sdtype': 'numerical'}},
 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}

In [ ]:
metadata.update_column(
    column_name='hypertension',
    sdtype='categorical')

metadata.update_column(
    column_name='heart_disease',
    sdtype='categorical')

metadata.update_column(
    column_name='stroke',
    sdtype='categorical')


In [ ]:
metadata.save_to_json(filepath='my_metadata_v6.json')

In [ ]:
metadata = SingleTableMetadata.load_from_json(
    filepath='my_metadata_v6.json')

In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer(
    metadata, # required
    enforce_min_max_values=True,
    enforce_rounding=False,
    numerical_distributions={       #'norm' 'beta', 'truncnorm', 'uniform', 'gamma' or 'gaussian_kde'
        'age': 'gaussian_kde',
        'bmi': 'norm',
        'avg_glucose_level':'gaussian_kde'
    },
    default_distribution='norm'
)
synthesizer.fit(data_stroke)

synthetic_data = synthesizer.sample(num_rows=10)


In [ ]:
# synthesizer.get_learned_distributions()

In [ ]:
# synthesizer = CTGANSynthesizer(
#     metadata, # required
#     enforce_rounding=False,
#     epochs=500,
#     verbose=True
# )

In [ ]:
synthesizer.fit(data_stroke)

In [ ]:
samples = synthesizer.sample(4000)
samples.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Female,78.835212,0,0,Yes,Private,Urban,70.950005,24.449498,never smoked,1
1,Male,56.876896,1,1,Yes,Private,Rural,248.931029,42.474226,never smoked,1
2,Male,73.567823,1,0,No,Private,Rural,183.869468,39.383721,formerly smoked,1
3,Female,48.296668,0,0,Yes,Private,Urban,131.386244,26.138939,Unknown,1
4,Female,61.624401,1,0,Yes,Self-employed,Rural,201.579401,30.494162,never smoked,1


In [ ]:
from sdv.evaluation.single_table  import evaluate_quality

quality_report = evaluate_quality(
    real_data=data_stroke,
    synthetic_data=samples,
    metadata=metadata)

Creating report: 100%|██████████| 4/4 [00:00<00:00,  6.16it/s]



Overall Quality Score: 90.12%

Properties:
Column Shapes: 93.0%
Column Pair Trends: 87.25%


In [ ]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic_report = run_diagnostic(
    real_data=data_stroke,
    synthetic_data=samples,
    metadata=metadata)

Creating report: 100%|██████████| 4/4 [00:02<00:00,  1.39it/s]


DiagnosticResults:

SUCCESS:
✓ The synthetic data covers over 90% of the numerical ranges present in the real data
✓ The synthetic data covers over 90% of the categories present in the real data
✓ Over 90% of the synthetic rows are not copies of the real data
✓ The synthetic data follows over 90% of the min/max boundaries set by the real data


In [ ]:
from sdv.evaluation.single_table import get_column_plot

fig= get_column_plot(
    real_data=data_stroke,
    synthetic_data=samples,
    column_name='age',
    metadata=metadata
)

fig.show()



In [ ]:
fig = get_column_plot(
    real_data=data_stroke,
    synthetic_data=samples,
    column_name='bmi',
    metadata=metadata
)

fig.show()

In [ ]:
fig = get_column_plot(
    real_data=data_stroke,
    synthetic_data=samples,
    column_name='avg_glucose_level',
    metadata=metadata
)

fig.show()

In [ ]:
# Combine data frames along the rows
combined_df = pd.concat([orig_data, samples], ignore_index=True)

# Shuffle the combined data frame randomly
combined_df = combined_df.sample(frac=1).reset_index(drop=True)

In [ ]:
combined_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,0.560000,0,0,No,children,Rural,57.020000,20.700000,Unknown,0
1,Male,51.000000,1,0,Yes,Private,Urban,76.100000,32.100000,smokes,0
2,Male,19.000000,0,0,No,Private,Rural,86.190000,26.200000,never smoked,0
3,Male,47.000000,0,0,Yes,Private,Rural,142.020000,30.000000,Unknown,0
4,Female,81.323056,1,1,No,Self-employed,Urban,108.525876,28.135116,formerly smoked,1


In [ ]:
print(len(combined_df.query("stroke==1")))

4209


In [ ]:
print(len(combined_df.query("stroke==0")))

4700


saving the file named as "healthcare_dataset_stroke_data_synthetic"

In [ ]:
combined_df.to_csv("healthcare_dataset_stroke_data_synthetic_v2.csv", index=False)
